In [2]:
import pandas as pd

In [31]:
url1 = "https://github.com/ThiesCalje/Survival-Analysis-Deep-Learning-AML-/blob/main/Data/Data_10_genex.xlsx?raw=true"
url2 = "https://github.com/ThiesCalje/Survival-Analysis-Deep-Learning-AML-/blob/main/Data/Table%2010.xlsx?raw=true"
gene_expression = pd.read_excel(url1).T.iloc[:-2,:]
patient_data = pd.read_excel(url2,index_col="Sample_ID")
patient_data, gene_expression

(                Treatment_Approach_(AML, ALL, Hybrid)  ...                               Reference
 Sample_ID                                              ...                                        
 SJAML001408_D1                                    AML  ...                         This Manuscript
 SJAML001427_D2                                    AML  ...                         This Manuscript
 SJAML001428_D1                                    AML  ...                         This Manuscript
 SJAML001430_D2                                    AML  ...                         This Manuscript
 SJAML001444_D1                                    AML  ...                         This Manuscript
 ...                                               ...  ...                                     ...
 SJALL015679_D1                                    ALL  ...     Nat Genet. 2017 Aug;49(8):1211-1218
 SJTALL022649_D2                                   ALL  ...     Nat Genet. 2017 Aug;49(8):1211-1218
